In [ ]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2
import vpype
from skimage import io
from pathlib import Path
import fn
import bezier

from sklearn.preprocessing import minmax_scale
from skimage import feature
from genpen.utils import Paper
from genpen.genpen import *
from genpen import subdivide as sd
from functools import partial
from genpen.grower import Grower, GrowerParams

In [ ]:
# make page
paper_size = 'A2'
border:float=45
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
drawbox.bounds

In [ ]:
drawbox

In [ ]:
split_func = functools.partial(sd.split_random_bezier, x0=0.2, x1=0.75, n_eval_points=50)

xgen = ss.uniform(loc=0.4, scale=0.01).rvs
split_func = functools.partial(sd.split_along_longest_side_of_min_rectangle, xgen=xgen)

# x0gen = ss.uniform(loc=0.15, scale=0.01).rvs
# x1gen = ss.uniform(loc=0.65, scale=0.01).rvs
# split_func = functools.partial(sd.split_random_line_gen, x0gen=x0gen, x1gen=x1gen)

In [ ]:
target = Point(140, 325)
target = drawbox.centroid

In [ ]:
dist_from_center = partial(sd.distance_from_pt, target=target, p_range=(0.99, 0.3,), d_range=(0, 200))
cp = sd.ContinuePolicy(dist_from_center)
polys = sd.very_flex_rule_recursive_split(poly=drawbox, split_func=split_func, continue_func=cp, depth_limit=14, buffer_kwargs={'distance':1e-6})


bps = gp.merge_Polygons(polys)

In [ ]:

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
n_layers = 1

In [ ]:
layers = []

In [ ]:

for ii in range(n_layers):
    fills = []
    for p in bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.005, 0.01)).rvs
        bhf = BezierHatchFill(
            spacing=np.random.uniform(0.2, 0.22),
            degrees=np.random.uniform(40,60),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=5,
            n_eval_points=6,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

for tolerance in [0.1, 0.3, 0.5, 0.7]:
    sk.vpype(f'linemerge --tolerance {tolerance}mm')
sk.vpype('linesimplify --tolerance 0.1 linesort')

sk.display(color_mode='layer')

In [ ]:
plot_id = fn.new_plot_id()

In [ ]:
savedir='/home/naka/art/plotter_svgs'

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

# more interesting initialization

In [ ]:
# make page
paper_size = 'A2'
border:float=45
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
params = GrowerParams(
    rad_func='loss_scaled_rad',
    rad_range=(70, 60),
    loss_range=(40, 100),
    n_pts_eval_per_iter=55,
    n_pts_add_per_iter=1,
    pt_to_poly_func='buffer_pt'
)
g = Grower(poly=drawbox.centroid.buffer(0.1), params=params)

In [ ]:
g.grow(6)

In [ ]:
# polys = g.mpoly.buffer(-5)
# polys = g.mpoly

polys = gp.make_like(g.mpoly, drawbox)

In [ ]:
bg = drawbox.difference(polys)

In [ ]:
polys = list(polys)
# polys.append(bg)

In [ ]:
# split_func = functools.partial(sd.split_random_bezier, x0=0.2, x1=0.75, n_eval_points=50)

xgen = ss.uniform(loc=0.5, scale=0.01).rvs
split_func = functools.partial(sd.split_along_longest_side_of_min_rectangle, xgen=xgen)

# x0gen = ss.uniform(loc=0.15, scale=0.01).rvs
# x1gen = ss.uniform(loc=0.65, scale=0.01).rvs
# split_func = functools.partial(sd.split_random_line_gen, x0gen=x0gen, x1gen=x1gen)

In [ ]:
target = Point(140, 325)
# target = drawbox.centroid

In [ ]:
xgen = ss.uniform(loc=0.4, scale=0.01).rvs
split_func = functools.partial(sd.split_along_longest_side_of_min_rectangle, xgen=xgen)
split_polys = []
for poly in polys:
    dist_from_center = partial(sd.distance_from_pt, target=target, p_range=(0.99, 0.3,), d_range=(0, 200))
    cp = sd.ContinuePolicy(dist_from_center)
    sps = sd.very_flex_rule_recursive_split(poly=poly, split_func=split_func, continue_func=cp, depth_limit=2, buffer_kwargs={'distance':1e-6})
    split_polys.append(sps)
    
xgen = ss.uniform(loc=0.5, scale=0.01).rvs
split_func = functools.partial(sd.split_along_longest_side_of_min_rectangle, xgen=xgen)
dist_from_center = partial(sd.distance_from_pt, target=target, p_range=(0.99, 0.3,), d_range=(0, 200))
cp = sd.ContinuePolicy(dist_from_center)
sps = sd.very_flex_rule_recursive_split(poly=bg, split_func=split_func, continue_func=cp, depth_limit=5, buffer_kwargs={'distance':1e-6})
split_polys.append(sps)


bps = gp.merge_Polygons(split_polys)

In [ ]:

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
n_layers = 1

In [ ]:
layers = []

In [ ]:

for ii in range(n_layers):
    fills = []
    for p in bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.001, 0.005)).rvs
        bhf = BezierHatchFill(
            spacing=np.random.uniform(0.2, 0.21),
            degrees=np.random.uniform(40,60),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=5,
            n_eval_points=6,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

for tolerance in [0.1, 0.3, 0.5, 0.7]:
    sk.vpype(f'linemerge --tolerance {tolerance}mm')
sk.vpype('linesimplify --tolerance 0.1 linesort')

sk.display(color_mode='layer')

In [ ]:
plot_id = fn.new_plot_id()

In [ ]:
savedir='/home/naka/art/plotter_svgs'

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

# more interesting initialization

In [ ]:
# make page
paper_size = '6x6 inches'
border:float=15
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
params = GrowerParams(
    rad_func='loss_scaled_rad',
    rad_range=(20, 10),
    loss_range=(20, 100),
    n_pts_eval_per_iter=55,
    n_pts_add_per_iter=1,
    pt_to_poly_func='reg_poly'
)
g = Grower(poly=drawbox.centroid.buffer(40), params=params)

In [ ]:
g.grow(2)

In [ ]:
# polys = g.mpoly.buffer(-1)
polys = g.mpoly

# polys = gp.make_like(g.mpoly, drawbox)

In [ ]:
bg = drawbox.difference(g.agg_poly)

In [ ]:
polys = gp.merge_Polygons(polys)
# polys.append(bg)

In [ ]:
# split_func = functools.partial(sd.split_random_bezier, x0=0.2, x1=0.75, n_eval_points=50)

xgen = ss.uniform(loc=0.5, scale=0.01).rvs
split_func = functools.partial(sd.split_along_longest_side_of_min_rectangle, xgen=xgen)

# x0gen = ss.uniform(loc=0.15, scale=0.01).rvs
# x1gen = ss.uniform(loc=0.65, scale=0.01).rvs
# split_func = functools.partial(sd.split_random_line_gen, x0gen=x0gen, x1gen=x1gen)

In [ ]:
# target = Point(140, 325)
target = drawbox.centroid

In [ ]:
split_polys =[]
    
x0gen = ss.uniform(loc=0.15, scale=0.01).rvs
x1gen = ss.uniform(loc=0.65, scale=0.1).rvs
split_func = functools.partial(sd.split_random_line_gen, x0gen=x0gen, x1gen=x1gen)
dist_from_center = partial(sd.distance_from_pt, target=target, p_range=(1, 0.3,), d_range=(0, 20))
cp = sd.ContinuePolicy(dist_from_center)

sps = sd.very_flex_rule_recursive_split(poly=drawbox, split_func=split_func, continue_func=cp, depth_limit=2, buffer_kwargs={'distance':1e-6})
sps = gp.merge_Polygons(sps)
split_polys.append(sps)


bps = gp.merge_Polygons(split_polys)

In [ ]:

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
n_layers = 1

In [ ]:
layers = []

In [ ]:

for ii in range(n_layers):
    fills = []
    for p in bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.001, 0.005)).rvs
        bhf = BezierHatchFill(
            spacing=np.random.uniform(0.2, 0.21),
            degrees=np.random.uniform(40,60),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=5,
            n_eval_points=6,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:

for ii in range(n_layers):
    fills = []
    for p in bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.001, 0.005)).rvs
        bhf = BezierHatchFill(
            spacing=np.random.uniform(0.2, 0.21),
            degrees=np.random.uniform(20,40),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=5,
            n_eval_points=6,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

for tolerance in [0.1, 0.3, 0.5, 0.7]:
    sk.vpype(f'linemerge --tolerance {tolerance}mm')
sk.vpype('linesimplify --tolerance 0.1 linesort')

sk.display(color_mode='layer')

In [ ]:
plot_id = fn.new_plot_id()

In [ ]:
savedir='/home/naka/art/plotter_svgs'

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)